## try out pytorch
We're going to try out pytorch to do some machine learning

TODO: remove NaN rows in the dataset. How do we do that


In [119]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
from matplotlib import pyplot as plt
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
    
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [124]:
class DryBeanDataset(Dataset):
    
    def __init__(self):
        # data loading
        #crop-yield-prediction-rs/Bean_Othello_Jul21_M10_transparent_reflectance_hyperindices.csv
        # import it as a pandas dataframe, slice it and change it to numpy array, 
#         xy = np.loadtxt('Bean_Othello_Jul21_M10_transparent_reflectance_hyperindices.csv', delimiter=',', dtype=np.float32, skiprows=1)
        xy = pd.read_csv('Bean_Othello_Jul21_M10_transparent_reflectance_hyperindices.csv', skiprows = 1).iloc[:, 10:].to_numpy().astype(np.float32)
#         s_sc = StandardScaler()
        self.x = torch.from_numpy(xy[:, 2:]) #gets all the index means
        self.y = torch.from_numpy(xy[:, 0]) # yield data
        
        
        
        
        #self.x = torch.from_numpy(xy.iloc[:, 12:].to_numpy().astype(np.float32)) #gets all the index means
        #self.y = torch.from_numpy(xy[:,11]) # 100 seed weight
        #self.y = torch.from_numpy(xy.iloc[:, 10].to_numpy().astype(np.float32)) # yield data
        #self.y = torch.from_numpy(xy[:, 9]) # harvest maturity
        #self.y = torch.from_numpy(xy[:, 8]) # emergence
        self.n_samples = xy.shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples
    
    
    

In [125]:
# create dataset
dataset = DryBeanDataset()
batch_size = 5
test_split = .2
shuffle_dataset = True
random_seed = 26

# Creating data indices for training and testing splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=test_sampler)

In [126]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out


In [128]:
# Hyper-parameters 
input_size = len(train_loader)
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        # resized: [5, 90]
        
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')



torch.Size([5, 90])


RuntimeError: shape '[-1, 784]' is invalid for input of size 450

In [129]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

RuntimeError: shape '[-1, 784]' is invalid for input of size 450